# Checkpoint 2 – An early explorer

Show us you can mine and gather insights from multiple data types. Submit  code that:

1. Loads two independent public sources (e.g., GEDI + TerraBrasilis polygons)
2. Produces at least five candidate “anomaly” footprints (bbox WKT or lat/lon center + radius)
3. Logs all dataset IDs and OpenAI prompts. Verify:  Automated script re‑runs → same five footprints ±50 m.
4. Show us how you can use this new data in future discovery - re-prompt the model with this leverage.

In [1]:
from xml.etree import ElementTree as ET
from shapely.geometry import Point
import logging, random, pathlib, datetime as dt
import geopandas as gpd
import numpy as np
import geemap

In [2]:
logging.basicConfig(
    filename="checkpoint2_run.log",
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
RUN_ID = dt.datetime.utcnow().isoformat(timespec="seconds")
pathlib.Path("outputs").mkdir(exist_ok=True)

/var/folders/1l/ts95kt9945j5dtyz_yqmlmpw0000gn/T/ipykernel_6977/2251294997.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = dt.datetime.utcnow().isoformat(timespec="seconds")


### Step 1: Load 2 Independent Data Sources
For our first dataset, we'll load TerraBrasilis.  We've already downloaded the main file from the [TerraBrasilis Website](https://terrabrasilis.dpi.inpe.br/en/download-files/).  Specifically, we're using "Complete PRODES in vector format - GeoPackage ~800MB (2007-2023)."

The main, unzipped file is far too large to load directly.  First, we'll do some data processing.  Mostly, this involves:
1. Focus only on recent data (in this case, 2023).
2. Take a random sampling of 10,000 polygons, instead of all of them.
3. Save the data as a .kml for easy access.

In [3]:
# Path to your KML file
tb_path = 'data/terrabrasilis/terrabrasilis_2023.0.kml'

# Finally, save the Terrabrasilis data as tb_gdf
tb_gdf = gpd.read_file(tb_path, driver='KML')

### Step 1 (Continued): Add the Second Data Set
For our second data set, we'll add the geoglyphs map provided by [James Q. Jacobs](https://www.jqjacobs.net/blog/).  For our purposes, we're focused on the main 'geoglyphs' layer.

In [4]:
# Parse with raw XML since fastkml is failing
gg_path = 'data/geoglyphs/amazon_geoglyphs.kml'
with open(gg_path, 'rt', encoding='utf-8') as f:
    doc = f.read()

root = ET.fromstring(doc)

# Get the Document element
document = root.find('.//{http://www.opengis.net/kml/2.2}Document')

# Get all Folders in the Document
folders = document.findall('.//{http://www.opengis.net/kml/2.2}Folder')

# Now extract Placemarks from all folders
all_placemarks = []
for i, folder in enumerate(folders):
    folder_name = folder.find('.//{http://www.opengis.net/kml/2.2}name')
    folder_name_text = folder_name.text if folder_name is not None else f"Folder_{i}"
    
    placemarks = folder.findall('.//{http://www.opengis.net/kml/2.2}Placemark')
    if folder_name_text == 'geoglyphs':
        all_placemarks.extend(placemarks)

print(f"Total geoglyphs found: {len(all_placemarks)}")

# Extract coordinates from placemarks
def extract_coordinates(placemark):
    """Extract lat/lon from a placemark"""
    # Look for Point geometry
    point = placemark.find('.//{http://www.opengis.net/kml/2.2}Point')
    if point is not None:
        coords = point.find('.//{http://www.opengis.net/kml/2.2}coordinates')
        if coords is not None:
            # KML coordinates are in format: lon,lat,alt (or just lon,lat)
            coord_text = coords.text.strip()
            lon, lat = coord_text.split(',')[:2]
            return float(lat), float(lon)
    
    # Look for other geometry types if needed
    return None, None

# Extract coordinates from all geoglyph placemarks
coordinates = []
names = []

for placemark in all_placemarks:  # your filtered geoglyphs placemarks
    lat, lon = extract_coordinates(placemark)
    if lat is not None and lon is not None:
        coordinates.append([lat, lon])
        
        # Get name if available
        name_elem = placemark.find('.//{http://www.opengis.net/kml/2.2}name')
        name = name_elem.text if name_elem is not None else "Unknown"
        names.append(name)

# Finally, save the geoglyph data as gg_gdf
gg_gdf = gpd.GeoDataFrame({
    'name': names,
    'geometry': [Point(lon, lat) for lat, lon in coordinates]
}, crs='EPSG:4326')

Total geoglyphs found: 1595


### Step 1 (Final): Visualize the Data
Finally, we extract the TerraBrasilis and Geoglyph data, and plot them on a map.

In [5]:
# Draw a map containing both the TerraBrasilis and geoglyph maps
centroid = gpd.GeoSeries(tb_gdf.geometry.tolist()).unary_union.centroid
Map = geemap.Map(center=(centroid.y, centroid.x), zoom=5)
Map.add_basemap('SATELLITE')
Map.add_gdf(gg_gdf, layer_name="Geoglyphs")
Map.add_kml(
    in_kml=tb_path,
    layer_name="TerraBrasilis, 2023",
    info_mode="on_hover",
    style={'color': 'red', 'weight': 2, 'fillOpacity': 0.1}
)
display(Map)

/var/folders/1l/ts95kt9945j5dtyz_yqmlmpw0000gn/T/ipykernel_6977/914529408.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gpd.GeoSeries(tb_gdf.geometry.tolist()).unary_union.centroid


Map(center=[-7.198098632959068, -57.68160332750702], controls=(WidgetControl(options=['position', 'transparent…

### Step 1: Conclusions
By finding areas where geoglyph and deforestation data overlap, we can identify areas that:
1. Are known to harbor large numbers of geoglyphs
2. Are unlikely to have been extensively searched by archaeologists.

### Steps 2 & 3: Produces at least five candidate “anomaly” footprints (lat/lon center) + Logs all OpenAI Prompts
Here are a few candidate anomalies that I found by scanning areas where the geoglyphs and deforestation areas overlap:

[9°38'8.15"S, 65°24'19.28"W] [link to chat](https://chatgpt.com/share/68388b55-c5a4-8004-a1bb-df06b70a1b09)

[9° 8'23.47"S, 65°23'59.25"W] [link to chat](https://chatgpt.com/share/68388d05-cb80-8004-bead-bfc62b93404b)

[8°43'59.96"S, 67° 2'51.32"W] [link to chat](https://chatgpt.com/share/68388ebc-9f08-8004-a3e1-e35217cc2232)

[9°22'36.48"S, 67°52'31.15"W] [link to chat](https://chatgpt.com/share/68388bde-38f8-8004-9486-a7ffd96d82e0)

[9°31'16.60"S, 67°53'20.68"W] [link to chat](https://chatgpt.com/share/68388cb6-0664-8004-8e66-22dc5856a156) (demonstrates a null result)

In [6]:
# Create points from the coordinates
points = [
    [-9.635597, -65.405356],  # 9°38'8.15"S, 65°24'19.28"W
    [-9.139853, -65.399792],  # 9° 8'23.47"S, 65°23'59.25"W
    [-8.733322, -67.047589],  # 8°43'59.96"S, 67° 2'51.32"W
    [-9.376800, -67.875319],  # 9°22'36.48"S, 67°52'31.15"W
    [-9.521278, -67.889078]   # 9°31'16.60"S, 67°53'20.68"W
]

# Create GeoDataFrame for the points
points_gdf = gpd.GeoDataFrame(
    geometry=[Point(lon, lat) for lat, lon in points],
    crs='EPSG:4326'
)

# Add points to the map with a distinct style
Map.add_gdf(
    points_gdf,
    layer_name="Candidate Points",
    style={
        'color': 'yellow',
        'weight': 3,
        'fillOpacity': 0.8,
        'radius': 8
    }
)

Map

Map(center=[np.float64(-6.39643196582367), np.float64(-65.98623650002926)], controls=(WidgetControl(options=['…